This notebook is to clean raw data downloaded by `0_collect_data`, steps include:
- Parsed min and max from salary range, calculated its average
- Converted hourly salary to annually
- Discarded data without salary (=-1)
- Made a new column for company state
- Made a new column if the job location is the same as head quarter
- Made a new column of company age
- Made new columns if selected skills listed in the job description
    - Python
    - Rstudio
    - SQL
    - Excel
    - AWS
    - Spark
    - NLP
- Made new columns for simplied job titles (scientist, analyst, engineer, etc) and seniority
- Made a new column for length of job description
- Made a new column for number of competitors
- Group "-1" into "Unknown" for 'Type of ownership','Sector','Size','Industry'
- Saved cleaned table into `DS_salary_cleaned.csv`

In [1]:
import pandas as pd

In [2]:
def title_simplifier(title):
    positions = ['data scientist','data engineer','analyst','machine learning','manager','director']
    for position in positions:
        if position in title.lower():
            return position
    return 'na'

def seniority(title):
    senior_strs = ['sr','sr.','senior','lead','principal']
    junior_strs = ['jr','jr.','junior','intern','co-op']
    if any(senior_str in title.lower() for senior_str in senior_strs):
        return 'senior'
    elif any(junior_str in title.lower() for junior_str in junior_strs):
        return 'junior'
    else:
        return 'na'

In [3]:
df = pd.read_csv('DS_salary_raw.csv')

# remove second column
df = df.drop(['Unnamed: 0'],axis=1)

# salary parsing, remove rows without salary info
df = df[df['Salary Estimate'] != '-1']

# add columns indicating if hourly paid
df['hourly'] = df['Salary Estimate'].str.lower().str.contains('per hour').astype(int)

# remove unnecessry strings and extract min, max and average salary, convert hourly to annualy
salary = df['Salary Estimate'].apply(lambda x: x.split('(')[0]).str.replace('$','').str.replace('K','').str.replace('Per Hour','').str.replace('Employer Provided Salary:','')
df['min_salary'] = salary.apply(lambda x: int(x.split('-')[0]))
df['max_salary'] = salary.apply(lambda x: int(x.split('-')[1]))
df['min_salary'] = df.apply(lambda x: x.min_salary * 2 if x.hourly == 1 else x.min_salary, axis = 1)
df['max_salary'] = df.apply(lambda x: x.max_salary * 2 if x.hourly == 1 else x.max_salary, axis = 1)
df['avg_salary'] = (df.min_salary + df.max_salary)/2

# company name
df['company_txt'] = df.apply(lambda x: x['Company Name'] if x['Rating'] < 0 else x['Company Name'][:-3], axis = 1)
df['company_txt'] = df['company_txt'].apply(lambda x: x.replace('\n',''))

# location by state and if head quarter in the same location
df['job_state'] = df['Location'].apply(lambda x: x.split(',')[1]).str.replace(' ','')
#df['job_state'].value_counts()
df['job_state'] = df['job_state'].str.replace('LosAngeles','CA')
df['same_state'] = df.apply(lambda x: 1 if x['Location'] == x['Headquarters'] else 0, axis = 1)

# age
df['age'] = df['Founded'].apply(lambda x: 2020 - int(x) if int(x) > 0 else int(x))

# job description keywords selection
skills = ['python','sql','excel','aws','spark','nlp','rstudio']

for skill in skills:
    df[skill + '_yn'] = df['Job Description'].apply(lambda x: 1 if skill in x.lower() else 0)
    #df[skill + '_yn'].value_counts()
    
# simply job title
df['job_simp'] = df['Job Title'].apply(title_simplifier)

# job seniority
df['seniority'] = df['Job Title'].apply(seniority)
df['seniority'].value_counts()

# length of job description
df['description_length'] = df['Job Description'].apply(len)

# number of competitors
df['num_competitor'] = df['Competitors'].apply(lambda x: len(x.split(',')) if x != -1 else 0)

# put missing rows into Unknown
miss_vars = ['Type of ownership','Sector','Size','Industry']
for miss_var in miss_vars:
    df[miss_var].replace({"-1":"Unknown"}, inplace=True)

In [4]:
#df.to_csv('DS_salary_cleaned.csv',index = False)